In [ ]:
import sys 
sys.path.append('../')
from src.utils.run_lib import * 

from src.core.passive_learning import *
from src.core.auto_labeling import *
from src.utils.logging_utils import * 
from src.datasets import dataset_factory  
from src.datasets.dataset_utils import * 
from src.utils.counting_utils import *  
from src.utils.common_utils import * 
from src.utils.vis_utils import *
import copy 
import random

from omegaconf import OmegaConf
#from calibration_utils import * 
from  src.datasets.data_manager import * 

from src.calibration.calibration_utils import * 

# configuration
conf_dir = '../configs/calib-exp/'

act_learn_conf_file = '{}/tiny_imagenet_CLIP_base_conf.yaml'.format(conf_dir)

#conf = load_yaml_config(pas_learn_conf_file)
conf = OmegaConf.load(act_learn_conf_file)

conf.data_conf.compute_emb = False 

#run_conf(conf)

In [ ]:
set_seed(conf['random_seed'])
dm = DataManager(conf,logger) # 11314 train, 7532 test

In [ ]:
len(dm.ds_std_train), len(dm.ds_std_val)

In [ ]:
(dm.ds_std_train[0][0]).shape

In [ ]:
# calib_conf:
#   type: 'post_hoc'
#   name: 'temp_scaling'
#   calib_val_frac: 0.1
#   training_conf: 
#     optimizer: 'adam'
#     learning_rate: 1 
#     batch_size: 1024
#     shuffle: True 
#     max_epochs: 100
#     normalize_weights: False 
#     log_train_ece: True 
#     log_train_ece_freq: 1 

from omegaconf import DictConfig
conf.calib_conf = DictConfig({
        "type": "post_hoc",
        "name": "temp_scaling",
        "calib_val_frac": 0.1,
        "training_conf": {
            "optimizer": "adam",
            "learning_rate": 0.5,
            "batch_size": 1024,
            "shuffle": True,
            "max_epochs": 100,
            "normalize_weights": False,
            "log_train_ece": True,
            "log_train_ece_freq": 1
            }
    })

In [ ]:

logger   = get_logger('../../../temp/logs/tiny_imagenet_act_learning.log',stdout_redirect=True,level=logging.DEBUG)

act_lbl = ActiveLabeling(conf,dm,logger)



In [ ]:
act_lbl.init()

lst_epoch_out = act_lbl.run_al_loop()

# logger.info('AL Loop Done')
# #test_err = al.get_test_error(al.cur_clf,test_set,conf['inference_conf'])
# out =  dm.get_auto_labeling_counts()

In [ ]:
# conf['train_pts_query_conf']['seed_train_size']= 10 
# conf['train_pts_query_conf']['max_num_train_pts']= 50

# conf['training_conf']['train_err_tol'] = -1
# conf['training_conf']['max_epochs'] = 50
# conf['training_conf']['weight_decay'] = 0.0
# conf['training_conf']['learning_rate'] = 1.0
# conf['training_conf']['use_lr_schedule'] = False 
# conf['training_conf']['optimizer_name'] = 'sgd'
# conf['training_conf']['batch_size'] = 32
# conf['training_conf']['momentum'] = 0.9
# conf['training_conf']['log_batch_loss_freq']=-1

In [ ]:

logger   = get_logger('../../../temp/logs/tiny_imagenet_passive_learning.log',stdout_redirect=True,level=logging.DEBUG)

set_seed(conf['random_seed'])

dm = DataManager(conf,logger)
len(dm.ds_std_train), len(dm.ds_std_val)


pl = PassiveLearning(conf,dm,logger)

out = pl.run()

w = pl.cur_clf.get_weights()
print(torch.norm(w))
test_err = get_test_error(pl.cur_clf,dm.ds_std_test,conf['inference_conf'])
print(test_err)

In [ ]:
cur_val_ds,cur_val_idcs=dm.get_current_validation_data()

val_set_subset = cur_val_ds

val_inf_out = pl.cur_clf.predict(val_set_subset,conf['inference_conf'])
cal_out = compute_calibration(val_set_subset.Y.numpy(), val_inf_out['labels'].numpy(), val_inf_out['confidence'].numpy(), num_bins=10)
print(cal_out['expected_calibration_error'])
ax = plt.subplot(111)
reliability_diagram_subplot(ax,cal_out)

plt.figure() 
o = val_inf_out

m = len(o['confidence'])
S = np.zeros((m,4))
S[:,0] = o['confidence']
S[:,1] = o['labels'] 
S[:,2] = val_set_subset.Y
S[:,3] = S[:,1]==S[:,2]

S = S[(-S[:,0]).argsort()]
 
bins = np.arange(0.1,1.1,0.05)
labels = ['corrects','incorrects']
plt.hist([S[S[:,3]==1,0],S[S[:,3]==0,0]],bins=bins,rwidth=0.5, histtype ='bar',label=labels)
plt.legend()
plt.ylabel('Count')
plt.xlabel('Confidence')
plt.grid() 
